# **Daily Minimum Temperatures Dataset**

## Hiểu dữ liệu (Dataset & Problem Definition)

### **Tải bộ dữ liệu**

In [ ]:
!gdown 1PWPwhW8QNOhPSOA1AtT7cu8-Uaxwo5UX

### **Import thư viện và load dataset**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Đọc và chuyển đổi dữ liệu Date sang dạng datetime

In [ ]:
DATASET_PATH = 'timeseries_daily-minimum-temperatures.csv'

In [ ]:
# Read data from .csv file
ts_df = pd.read_csv(DATASET_PATH, parse_dates=["Date"])

print(ts_df.shape)
print(ts_df.dtypes)
ts_df.head(3)

Dữ liệu có 3650 dòng, 2 cột

In [ ]:
ts_df.info()

- Không có dòng dữ liệu nào bị thiếu giá trị
- Không có dòng bị trùng lặp

In [ ]:
print("Missing values:", ts_df.isnull().sum().values[0])
print("Duplicate rows:", ts_df.duplicated().sum())

In [ ]:
ts_df['Date'] = pd.to_datetime(ts_df['Date'])
# ts_df = ts_df.set_index('Date')
ts_df.head(3)

In [ ]:
ts_df = pd.read_csv(DATASET_PATH, index_col=0, parse_dates=True)
ts_df['Year'] = ts_df.index.year
ts_df['Month'] = ts_df.index.month
ts_df['Weekday Name'] = ts_df.index.day_name()

ts_df.sample(5, random_state=0)

In [ ]:
ts_df.dtypes

In [ ]:

ts_df['Daily minimum temperatures'] = pd.to_numeric(
    ts_df['Daily minimum temperatures'], errors='coerce')
ts_df.dtypes

In [ ]:
ts_df.describe()

### **Time-based indexing**

In [ ]:
ts_df = pd.read_csv(DATASET_PATH, index_col=0, parse_dates=True)

# Buộc cột nhiệt độ về dạng số (float), chuyển các giá trị không hợp lệ (như chuỗi header bị sót) thành NaN.
ts_df['Daily minimum temperatures'] = pd.to_numeric(
    ts_df['Daily minimum temperatures'],
    errors='coerce'
)

In [ ]:
ts_df.loc['1990-01-01':'1991-01-01']

In [ ]:
ts_df.loc['1990-01']

### **Frequencies**

In [ ]:
pd.date_range('1998-03-10', '1998-03-15', freq='D')

In [ ]:
pd.date_range('2004-09-20', periods=8, freq='h')

In [ ]:
# To select an arbitrary sequence of date/time values from a pandas time series,
# we need to use a DatetimeIndex, rather than simply a list of date/time strings
times_sample = pd.to_datetime(['1981-01-01', '1981-01-04', '1981-01-08'])
# Select the specified dates and just the Consumption column
consum_sample = ts_df.loc[times_sample, ['Daily minimum temperatures']].copy()
consum_sample

In [ ]:
# Convert the data to daily frequency, without filling any missings
consum_freq = consum_sample.asfreq('D')
# Create a column with missings forward filled
consum_freq['Daily minimum temperatures - Forward Fill'] = consum_sample.asfreq(
    'D', method='ffill')
consum_freq

## Khám phá dữ liệu (Exploratory Data Analysis) và Feature Engineering

In [ ]:
from matplotlib.dates import YearLocator, DateFormatter

# ==============================================================================
# THIẾT LẬP CẤU HÌNH TOÀN CỤC (GLOBAL CONFIGURATION)
# Mục đích: Tập trung tất cả các tham số của Mô hình Sinusoidal và Phân tích
# ==============================================================================
CONFIG = {
    # Cấu hình Mô hình Sinusoidal: y = A * sin(b*x + c) + d

    # BASE_TEMP_D (d): Nhiệt độ trung bình/Độ cao cơ sở hàng năm tại Melbourne: https://en.wikipedia.org/wiki/Climate_of_Melbourne
    "BASE_TEMP_D": 9.7,

    # AMPLITUDE_A (A): Biên độ dao động (Lý thuyết)
    # Dữ liệu lấy từ Wikipedia (Trung bình tối thiểu hàng tháng tại Sân bay Melbourne):
    # Nhiệt độ Min Max (Tháng 2) = 14.4 °C; Nhiệt độ Min Min (Tháng 7) = 5.8 °C.
    # Chênh lệch thực tế là (14.4 - 5.8) = 8.6 °C. Giá trị 8 °C được chọn để làm tròn và làm mượt biên độ này.
    "AMPLITUDE_A": 8,

    # DAYS_IN_YEAR (T): Chu kỳ của hàm Sin (365.25 ngày/năm)
    "DAYS_IN_YEAR": 365.25,

    # DAYS_TO_SHIFT_PEAK (c - tính bằng ngày): Dịch pha
    # Giá trị 60 ngày được sử dụng để dịch chuyển đỉnh sóng Sin (mùa nóng nhất) từ
    # vị trí mặc định (khoảng Tháng 4) về Tháng 2, phù hợp với dữ liệu khí hậu thực tế.
    "DAYS_TO_SHIFT_PEAK": 60,

    # Cấu hình Kỹ thuật Phân tích
    "ROLLING_WINDOW_DAYS": 30,      # Cửa sổ (số ngày) tính Trung bình Trượt

    # Cấu hình Ngôn ngữ/Hiển thị
    "MONTH_NAMES": {
        1: 'Tháng 1', 2: 'Tháng 2', 3: 'Tháng 3', 4: 'Tháng 4',
        5: 'Tháng 5', 6: 'Tháng 6', 7: 'Tháng 7', 8: 'Tháng 8',
        9: 'Tháng 9', 10: 'Tháng 10', 11: 'Tháng 11', 12: 'Tháng 12'
    }
}

# ------------------------------------------------------------------------------
print("\n--- 1. TÍNH TOÁN CÁC HẰNG SỐ DẪN XUẤT (DERIVED CONSTANTS) ---")
# Các tham số được chuyển đổi sang đơn vị radian cho hàm np.sin()

# ANGULAR_FREQUENCY_B (b): Tần số góc = 2*pi / Chu kỳ (365.25)
CONFIG["ANGULAR_FREQUENCY_B"] = (2 * np.pi) / CONFIG["DAYS_IN_YEAR"]
print(f"Tần số góc (b): {CONFIG['ANGULAR_FREQUENCY_B']:.6f}")

# PHASE_SHIFT_RADIAN_C (c - tính bằng radian): Dịch pha
CONFIG["PHASE_SHIFT_RADIAN_C"] = (2 * np.pi * CONFIG["DAYS_TO_SHIFT_PEAK"]) / CONFIG["DAYS_IN_YEAR"]
print(f"Dịch pha (c): {CONFIG['PHASE_SHIFT_RADIAN_C']:.6f} radians")

In [ ]:
# ==============================================================================
print("\n--- 2. PHẦN 1: DỮ LIỆU THỰC VÀ TÍNH TOÁN MÔ HÌNH ---")
# ==============================================================================

# 1. DỮ LIỆU THỰC TẾ
ts_df = pd.read_csv(DATASET_PATH, index_col=0, parse_dates=True)

# Buộc cột nhiệt độ về dạng số (float), chuyển các giá trị không hợp lệ (như chuỗi header bị sót) thành NaN.
ts_df['Daily minimum temperatures'] = pd.to_numeric(
    ts_df['Daily minimum temperatures'],
    errors='coerce'
)
# Loại bỏ các dòng bị NaN sau khi ép kiểu, đảm bảo chỉ còn dữ liệu số hợp lệ.
ts_df = ts_df.dropna(subset=['Daily minimum temperatures'])

### Feature Engineering

In [ ]:
# 2. TÍNH TOÁN VÀ THÊM MÔ HÌNH SINUSOIDAL (SIGNAL) VÀO ts_df
# Chỉ số thời gian (x): Dựa trên độ dài dữ liệu thực tế đã được tải
time_index = np.arange(len(ts_df))

# Tính toán thành phần tín hiệu PURE SIGNAL MODEL (Sóng Sin)
model_signal = (
    CONFIG["BASE_TEMP_D"] +
    CONFIG["AMPLITUDE_A"] * np.sin(
        CONFIG["ANGULAR_FREQUENCY_B"] * time_index +
        CONFIG["PHASE_SHIFT_RADIAN_C"]
    )
)
print(f"Đã tính toán Mô hình Sinusoidal")

# Thêm cột Signal vào DataFrame đã tải để so sánh trực quan
ts_df["Signal"] = model_signal
print(f"Đã thêm cột Signal (Mô hình Lý thuyết) vào DataFrame ts_df.")

# ==============================================================================
print("\n--- 3. PHẦN 2: PHÂN TÍCH CHUỖI THỜI GIAN ---")
print("Kỹ thuật: Trung bình Trượt (Rolling Mean)")
# ==============================================================================
# Tính Trung bình Trượt trên DỮ LIỆU THỰC TẾ
ts_df['Rolling 30-Day Avg'] = ts_df['Daily minimum temperatures'].rolling(
    window=CONFIG["ROLLING_WINDOW_DAYS"],
    center=True
).mean()
print(f"Đã tính Trung bình Trượt {CONFIG['ROLLING_WINDOW_DAYS']}-Ngày cho Dữ liệu Thực tế.")

In [ ]:
# ==============================================================================
print("\n--- 4. PHẦN 3: PHÂN TÍCH THEO MÙA (MONTHLY ANALYSIS) ---")
# ==============================================================================

# Thực hiện phân tích trên DỮ LIỆU THỰC TẾ
ts_df['Month'] = ts_df.index.month
monthly_avg = ts_df.groupby('Month')['Daily minimum temperatures'].mean().reset_index()
monthly_avg['Month Name'] = monthly_avg['Month'].map(CONFIG["MONTH_NAMES"])
max_temp_month = monthly_avg.loc[monthly_avg['Daily minimum temperatures'].idxmax()]
min_temp_month = monthly_avg.loc[monthly_avg['Daily minimum temperatures'].idxmin()]
print("Đã thực hiện nhóm dữ liệu theo Tháng và tính Trung bình mỗi tháng trên DỮ LIỆU THỰC TẾ.")

# In kết quả phân tích ra Console
print("\n--- KẾT QUẢ PHÂN TÍCH NHIỆT ĐỘ TRUNG BÌNH THEO THÁNG (Dữ liệu Thực tế) ---")
print(f"Tháng NÓNG NHẤT (trung bình): {max_temp_month['Month Name']} ({max_temp_month['Daily minimum temperatures']:.2f} °C)")
print(f"Tháng LẠNH NHẤT (trung bình): {min_temp_month['Month Name']} ({min_temp_month['Daily minimum temperatures']:.2f} °C)")
print("---------------------------------------------------------------")

In [ ]:
ts_df.describe()

### Phân tích Tính thời vụ (Seasonality Analysis) - EDA step

In [ ]:
# Thiết lập biểu đồ
plt.figure(figsize=(14, 6))

# VẼ 1: DỮ LIỆU THỰC TẾ (Đọc từ CSV)
plt.plot(
    ts_df.index,
    ts_df["Daily minimum temperatures"],
    color="#0066CC",
    linewidth=1.0,
    alpha=0.6,
    label="1. Dữ liệu Thực tế (CSV)"
)

# VẼ 2: MÔ HÌNH LÝ THUYẾT (PURE SIGNAL) - Đường tham chiếu Sóng Sin
plt.plot(
    ts_df.index,
    ts_df["Signal"],
    color="#000000",
    linewidth=1.5,
    linestyle='-',
    label="2. Mô hình Sinusoidal (Lý thuyết)"
)

# VẼ 3: Trung bình trượt (Smoothed Data) - Làm mượt Dữ liệu Thực tế
plt.plot(
    ts_df.index,
    ts_df["Rolling 30-Day Avg"],
    color="#CC0000",
    linewidth=1.0,
    linestyle='--',
    label=f"3. Trung bình trượt {CONFIG['ROLLING_WINDOW_DAYS']}-Ngày"
)

# Cải thiện Trục X (Thời gian)
ax = plt.gca()
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter('%Y'))
plt.gcf().autofmt_xdate()

# Tiêu đề và chú thích
plt.title(
    "So sánh Dữ liệu Thực tế (CSV) và Mô hình Sinusoidal Tinh khiết",
    fontsize=16,
    fontweight='bold'
)
plt.xlabel("Năm", fontsize=12)
plt.ylabel("Nhiệt độ (°C)", fontsize=12)
plt.legend(loc='upper right', frameon=True)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
print("Đã vẽ Biểu đồ Chuỗi Thời gian (Dữ liệu Thực tế và Mô hình).")

In [ ]:
# VẼ BIỂU ĐỒ THANH CHO PHÂN TÍCH THEO THÁNG
plt.figure(figsize=(10, 4))
color_map = ['tab:red' if m == max_temp_month['Month Name'] else 'tab:blue' if m == min_temp_month['Month Name'] else 'tab:grey' for m in monthly_avg['Month Name']]

plt.bar(
    monthly_avg['Month Name'],
    monthly_avg['Daily minimum temperatures'],
    color=color_map
)
plt.title("Nhiệt độ Tối thiểu Trung bình theo Tháng (Dữ liệu Thực tế)", fontsize=14)
plt.xlabel("Tháng", fontsize=12)
plt.ylabel("Nhiệt độ TB (°C)", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
print("Đã vẽ Biểu đồ Thanh phân tích theo Tháng.")

# Hiển thị tất cả biểu đồ
plt.show()


Minh họa dữ liệu theo từng năm 

In [ ]:
ts_df = pd.read_csv(DATASET_PATH, index_col=0, parse_dates=True)

# Buộc cột nhiệt độ về dạng số (float), chuyển các giá trị không hợp lệ (như chuỗi header bị sót) thành NaN.
ts_df['Daily minimum temperatures'] = pd.to_numeric(
    ts_df['Daily minimum temperatures'],
    errors='coerce'
)

sns.set(rc={'figure.figsize':(9, 4)})
col_to_plot = 'Daily minimum temperatures'
ts_df[col_to_plot].plot(linewidth=0.8)
plt.ylabel(col_to_plot)
plt.show()

In [ ]:
cols_plot = ['Daily minimum temperatures']
ts_df[cols_plot].plot(
    marker='.',
    alpha=0.5,
    linestyle='None',
    figsize=(10, 4),
    subplots=True
)
plt.ylabel('Daily minimum temperatures')
plt.show()

### **Phát hiện outliers**



In [ ]:
ts_df = pd.read_csv(DATASET_PATH, index_col=0, parse_dates=True)
ts_df['Year'] = ts_df.index.year
ts_df['Month'] = ts_df.index.month
ts_df['Weekday Name'] = ts_df.index.day_name()
# Buộc cột nhiệt độ về dạng số (float), chuyển các giá trị không hợp lệ (như chuỗi header bị sót) thành NaN.
ts_df['Daily minimum temperatures'] = pd.to_numeric(
    ts_df['Daily minimum temperatures'],
    errors='coerce'
)
COL_NAME = 'Daily minimum temperatures'
plt.figure(figsize=(10, 4))
sns.boxplot(data=ts_df, x='Month', y=COL_NAME)
plt.ylabel('Temperature')
plt.title(COL_NAME)
plt.show()

### **Resampling - Feature engineering** 

In [ ]:
data_columns = ['Daily minimum temperatures']
ts_weekly_mean = ts_df[data_columns].resample('W').mean()
ts_weekly_mean.head(3)

In [ ]:
print(ts_df.shape[0])
print(ts_weekly_mean.shape[0])

In [ ]:
# Start and end of the date range to extract
start, end = '1981-01', '1981-12'
# Plot daily and weekly resampled time series together
fig, ax = plt.subplots()
ax.plot(ts_df.loc[start:end, 'Daily minimum temperatures'],
marker='.', linestyle='-', linewidth=0.5, label='Daily')
ax.plot(ts_weekly_mean.loc[start:end, 'Daily minimum temperatures'],
marker='o', linestyle='-', label='Weekly Mean Resample')
ax.set_ylabel('Temperature)')
ax.legend()
plt.show()

In [ ]:
# Compute the annual sums, setting the value to NaN for any year which has
# fewer than 360 days of data
ts_annual_df = ts_df[data_columns].resample('A').sum(min_count=360)
# The default index of the resampled DataFrame is the last day of each year
# to the year component
ts_annual_df = ts_annual_df.set_index(ts_annual_df.index.year)
ts_annual_df.index.name = 'Year'
# Compute the ratio of Wind+Solar to Consumption
ts_annual_df.tail(3)

In [ ]:
# Plot from 1981 onwards, because there is no solar production data in earlier years
ax = ts_annual_df.loc[1981:, 'Daily minimum temperatures'].plot.bar(color='C0')
ax.set_ylabel('Fraction')
ax.set_title('Annual Temperature')
plt.xticks(rotation=0)
plt.show()

### Chuẩn bị dữ liệu cho training

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Đọc dữ liệu
df = pd.read_csv('timeseries_daily-minimum-temperatures.csv')

print("===== KIỂM TRA DỮ LIỆU BAN ĐẦU =====")
print(f"Shape: {df.shape}")
print(f"\nCác cột trong dataset:")
print(df.columns.tolist())
print(f"\nThông tin các cột:")
print(df.info())
print(f"\n5 dòng đầu:")
print(df.head())
print(f"\nKiểu dữ liệu:")
print(df.dtypes)

# Tự động tìm tên cột nhiệt độ (cột số không phải Date)
date_col = None
temp_col = None

for col in df.columns:
    col_lower = col.lower()
    if 'date' in col_lower:
        date_col = col
    elif df[col].dtype in ['float64', 'int64', 'float32', 'int32']:
        temp_col = col

# Nếu không tìm thấy, thử cột thứ 2
if temp_col is None:
    temp_col = df.columns[1]
if date_col is None:
    date_col = df.columns[0]

print(f"\n✓ Cột Date: {date_col}")
print(f"✓ Cột Temperature: {temp_col}")

# Chuyển đổi kiểu dữ liệu
df[date_col] = pd.to_datetime(df[date_col])
df[temp_col] = pd.to_numeric(df[temp_col], errors='coerce')

# Loại bỏ missing values
df = df.dropna()
df = df.sort_values(date_col)
df = df.reset_index(drop=True)

print(f"\nShape sau khi xử lý: {df.shape}")
print(f"Thống kê nhiệt độ:")
print(df[temp_col].describe())

# ===== FEATURE ENGINEERING =====

def create_time_series_features(df, target_col, date_col):
    """
    Tạo các features cho time series
    """
    df_features = df.copy()

    # 1. TIME-BASED FEATURES (Đặc trưng thời gian)
    df_features['year'] = df_features[date_col].dt.year
    df_features['month'] = df_features[date_col].dt.month
    df_features['day'] = df_features[date_col].dt.day
    df_features['dayofweek'] = df_features[date_col].dt.dayofweek
    df_features['dayofyear'] = df_features[date_col].dt.dayofyear
    df_features['quarter'] = df_features[date_col].dt.quarter
    df_features['weekofyear'] = df_features[date_col].dt.isocalendar().week.astype(int)

    # 2. CYCLIC FEATURES (Đặc trưng tuần hoàn)
    df_features['month_sin'] = np.sin(2 * np.pi * df_features['month'] / 12)
    df_features['month_cos'] = np.cos(2 * np.pi * df_features['month'] / 12)
    df_features['day_sin'] = np.sin(2 * np.pi * df_features['day'] / 31)
    df_features['day_cos'] = np.cos(2 * np.pi * df_features['day'] / 31)
    df_features['dayofyear_sin'] = np.sin(2 * np.pi * df_features['dayofyear'] / 365)
    df_features['dayofyear_cos'] = np.cos(2 * np.pi * df_features['dayofyear'] / 365)

    # 3. LAG FEATURES (Giá trị quá khứ)
    for lag in [1, 2, 3, 7, 14, 30]:
        df_features[f'temp_lag_{lag}'] = df_features[target_col].shift(lag)

    # 4. ROLLING WINDOW FEATURES (Đặc trưng cửa sổ trượt)
    for window in [3, 7, 14, 30]:
        df_features[f'temp_rolling_mean_{window}'] = df_features[target_col].rolling(window=window, min_periods=1).mean()
        df_features[f'temp_rolling_std_{window}'] = df_features[target_col].rolling(window=window, min_periods=1).std()
        df_features[f'temp_rolling_min_{window}'] = df_features[target_col].rolling(window=window, min_periods=1).min()
        df_features[f'temp_rolling_max_{window}'] = df_features[target_col].rolling(window=window, min_periods=1).max()

    # 5. EXPANDING WINDOW FEATURES (Đặc trưng mở rộng)
    df_features['temp_expanding_mean'] = df_features[target_col].expanding(min_periods=1).mean()
    df_features['temp_expanding_std'] = df_features[target_col].expanding(min_periods=1).std()

    # 6. DIFFERENCE FEATURES (Đặc trưng sự thay đổi)
    df_features['temp_diff_1'] = df_features[target_col].diff(1)
    df_features['temp_diff_7'] = df_features[target_col].diff(7)

    # 7. EXPONENTIAL WEIGHTED FEATURES
    df_features['temp_ewm_mean_7'] = df_features[target_col].ewm(span=7, min_periods=1).mean()
    df_features['temp_ewm_std_7'] = df_features[target_col].ewm(span=7, min_periods=1).std()

    return df_features

# Tạo features
print("\n===== TẠO FEATURES =====")
df_features = create_time_series_features(df, temp_col, date_col)

print(f"Số lượng features: {len(df_features.columns)}")
print(f"\nDanh sách features:")
for i, col in enumerate(df_features.columns, 1):
    print(f"{i}. {col}")

# Xử lý missing values
print(f"\n\nMissing values:")
missing_counts = df_features.isnull().sum()
if missing_counts.sum() > 0:
    print(missing_counts[missing_counts > 0].sort_values(ascending=False))
    df_features = df_features.dropna()
    print(f"\nShape sau khi xử lý missing values: {df_features.shape}")
else:
    print("Không có missing values!")

# ===== CHUẨN BỊ DỮ LIỆU CHO TRAINING =====

print(f"\n{'='*70}")
print("PHÂN TÁCH FEATURES VÀ TARGET")
print(f"{'='*70}")

# Tách features và target
feature_columns = [col for col in df_features.columns if col not in [date_col, temp_col]]
X_features = df_features[feature_columns].copy()
y_target = df_features[temp_col].copy()

print(f"\n📊 FEATURES (X_features):")
print(f"   Shape: {X_features.shape}")
print(f"   Số lượng features: {len(feature_columns)}")
print(f"\n   5 dòng đầu tiên:")
print(X_features.head())

print(f"\n📊 TARGET (y_target):")
print(f"   Shape: {y_target.shape}")
print(f"   Tên cột: '{temp_col}'")
print(f"\n   5 giá trị đầu tiên:")
print(y_target.head())

print(f"\n📈 Thống kê y_target:")
print(y_target.describe())

# ===== CHIA TRAIN/TEST THEO THỜI GIAN =====

print(f"\n{'='*70}")
print("PHÂN CHIA DỮ LIỆU THEO THỜI GIAN")
print(f"{'='*70}")

# Chia train/test theo thời gian (80/20)
split_idx = int(len(X_features) * 0.8)
X_train, X_test = X_features.iloc[:split_idx], X_features.iloc[split_idx:]
y_train, y_test = y_target.iloc[:split_idx], y_target.iloc[split_idx:]

# Lấy ngày tháng tương ứng
train_start = df_features[date_col].iloc[0]
train_end = df_features[date_col].iloc[split_idx-1]
test_start = df_features[date_col].iloc[split_idx]
test_end = df_features[date_col].iloc[-1]

print(f"\n📊 Kích thước:")
print(f"   Train set: {X_train.shape[0]:,} mẫu ({X_train.shape[0]/len(X_features)*100:.1f}%)")
print(f"   Test set:  {X_test.shape[0]:,} mẫu ({X_test.shape[0]/len(X_features)*100:.1f}%)")

print(f"\n📅 Khoảng thời gian TRAIN:")
print(f"   Từ: {train_start.strftime('%Y-%m-%d')}")
print(f"   Đến: {train_end.strftime('%Y-%m-%d')}")

print(f"\n📅 Khoảng thời gian TEST:")
print(f"   Từ: {test_start.strftime('%Y-%m-%d')}")
print(f"   Đến: {test_end.strftime('%Y-%m-%d')}")

# Tính số ngày
train_days = (train_end - train_start).days + 1
test_days = (test_end - test_start).days + 1
print(f"\n⏱️  Độ dài:")
print(f"   Train: {train_days:,} ngày")
print(f"   Test:  {test_days:,} ngày")

# ===== CHUẨN HÓA DỮ LIỆU =====

print(f"\n{'='*70}")
print("CHUẨN HÓA DỮ LIỆU (STANDARDIZATION)")
print(f"{'='*70}")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Chuyển về DataFrame để giữ tên cột
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_columns, index=X_test.index)

print(f"\n✓ X_train_scaled shape: {X_train_scaled.shape}")
print(f"✓ X_test_scaled shape: {X_test_scaled.shape}")

print(f"\n📊 Mẫu dữ liệu X_train_scaled (5 dòng đầu):")
print(X_train_scaled.head())

print(f"\n📊 Thống kê sau chuẩn hóa (X_train_scaled):")
print(X_train_scaled.describe().loc[['mean', 'std', 'min', 'max']])

# ===== KIỂM TRA CHẤT LƯỢNG DỮ LIỆU =====

print(f"\n{'='*70}")
print("KIỂM TRA CHẤT LƯỢNG DỮ LIỆU")
print(f"{'='*70}")

# Kiểm tra NaN
nan_train = X_train_scaled.isnull().sum().sum()
nan_test = X_test_scaled.isnull().sum().sum()
nan_y_train = y_train.isnull().sum()
nan_y_test = y_test.isnull().sum()

print(f"\n🔍 Missing Values (NaN):")
print(f"   X_train_scaled: {nan_train}")
print(f"   X_test_scaled: {nan_test}")
print(f"   y_train: {nan_y_train}")
print(f"   y_test: {nan_y_test}")

# Kiểm tra Inf
inf_train = np.isinf(X_train_scaled.values).sum()
inf_test = np.isinf(X_test_scaled.values).sum()

print(f"\n🔍 Infinite Values (Inf):")
print(f"   X_train_scaled: {inf_train}")
print(f"   X_test_scaled: {inf_test}")

if nan_train + nan_test + inf_train + inf_test + nan_y_train + nan_y_test == 0:
    print(f"\n✅ DỮ LIỆU SẠCH - Không có NaN hoặc Inf!")
else:
    print(f"\n⚠️  CẢNH BÁO: Dữ liệu có vấn đề cần xử lý!")

# ===== TỔNG KẾT =====

print(f"\n{'='*70}")
print("✅ FEATURE ENGINEERING HOÀN THÀNH")
print(f"{'='*70}")

print("\n📦 CÁC BIẾN ĐÃ TẠO:")
print(f"   1. X_features: DataFrame chứa {X_features.shape[1]} features gốc")
print(f"   2. y_target: Series chứa target '{temp_col}'")
print(f"   3. X_train_scaled: Features đã chuẩn hóa cho training ({X_train_scaled.shape})")
print(f"   4. X_test_scaled: Features đã chuẩn hóa cho testing ({X_test_scaled.shape})")
print(f"   5. y_train: Target cho training ({y_train.shape})")
print(f"   6. y_test: Target cho testing ({y_test.shape})")
print(f"   7. scaler: StandardScaler object (để transform dữ liệu mới)")
print(f"   8. df_features: DataFrame đầy đủ với tất cả features")

print(f"\n📌 TÊN CỘT:")
print(f"   - temp_col = '{temp_col}'")
print(f"   - date_col = '{date_col}'")

print(f"\n{'='*70}\n")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Đọc dữ liệu
df = pd.read_csv('timeseries_daily-minimum-temperatures.csv')

print("===== KIỂM TRA DỮ LIỆU BAN ĐẦU =====")
print(f"Shape: {df.shape}")
print(f"\nCác cột trong dataset:")
print(df.columns.tolist())
print(f"\nThông tin các cột:")
print(df.info())
print(f"\n5 dòng đầu:")
print(df.head())
print(f"\nKiểu dữ liệu:")
print(df.dtypes)

# Tự động tìm tên cột nhiệt độ (cột số không phải Date)
date_col = None
temp_col = None

for col in df.columns:
    col_lower = col.lower()
    if 'date' in col_lower:
        date_col = col
    elif df[col].dtype in ['float64', 'int64', 'float32', 'int32']:
        temp_col = col

# Nếu không tìm thấy, thử cột thứ 2
if temp_col is None:
    temp_col = df.columns[1]
if date_col is None:
    date_col = df.columns[0]

print(f"\n✓ Cột Date: {date_col}")
print(f"✓ Cột Temperature: {temp_col}")

# Chuyển đổi kiểu dữ liệu
df[date_col] = pd.to_datetime(df[date_col])
df[temp_col] = pd.to_numeric(df[temp_col], errors='coerce')

# Loại bỏ missing values
df = df.dropna()
df = df.sort_values(date_col)
df = df.reset_index(drop=True)

print(f"\nShape sau khi xử lý: {df.shape}")
print(f"Thống kê nhiệt độ:")
print(df[temp_col].describe())

# ===== FEATURE ENGINEERING =====

def create_time_series_features(df, target_col, date_col):
    """
    Tạo các features cho time series
    """
    df_features = df.copy()

    # 1. TIME-BASED FEATURES (Đặc trưng thời gian)
    df_features['year'] = df_features[date_col].dt.year
    df_features['month'] = df_features[date_col].dt.month
    df_features['day'] = df_features[date_col].dt.day
    df_features['dayofweek'] = df_features[date_col].dt.dayofweek
    df_features['dayofyear'] = df_features[date_col].dt.dayofyear
    df_features['quarter'] = df_features[date_col].dt.quarter
    df_features['weekofyear'] = df_features[date_col].dt.isocalendar().week.astype(int)

    # 2. CYCLIC FEATURES (Đặc trưng tuần hoàn)
    df_features['month_sin'] = np.sin(2 * np.pi * df_features['month'] / 12)
    df_features['month_cos'] = np.cos(2 * np.pi * df_features['month'] / 12)
    df_features['day_sin'] = np.sin(2 * np.pi * df_features['day'] / 31)
    df_features['day_cos'] = np.cos(2 * np.pi * df_features['day'] / 31)
    df_features['dayofyear_sin'] = np.sin(2 * np.pi * df_features['dayofyear'] / 365)
    df_features['dayofyear_cos'] = np.cos(2 * np.pi * df_features['dayofyear'] / 365)

    # 3. LAG FEATURES (Giá trị quá khứ)
    for lag in [1, 2, 3, 7, 14, 30]:
        df_features[f'temp_lag_{lag}'] = df_features[target_col].shift(lag)

    # 4. ROLLING WINDOW FEATURES (Đặc trưng cửa sổ trượt)
    for window in [3, 7, 14, 30]:
        df_features[f'temp_rolling_mean_{window}'] = df_features[target_col].rolling(window=window, min_periods=1).mean()
        df_features[f'temp_rolling_std_{window}'] = df_features[target_col].rolling(window=window, min_periods=1).std()
        df_features[f'temp_rolling_min_{window}'] = df_features[target_col].rolling(window=window, min_periods=1).min()
        df_features[f'temp_rolling_max_{window}'] = df_features[target_col].rolling(window=window, min_periods=1).max()

    # 5. EXPANDING WINDOW FEATURES (Đặc trưng mở rộng)
    df_features['temp_expanding_mean'] = df_features[target_col].expanding(min_periods=1).mean()
    df_features['temp_expanding_std'] = df_features[target_col].expanding(min_periods=1).std()

    # 6. DIFFERENCE FEATURES (Đặc trưng sự thay đổi)
    df_features['temp_diff_1'] = df_features[target_col].diff(1)
    df_features['temp_diff_7'] = df_features[target_col].diff(7)

    # 7. EXPONENTIAL WEIGHTED FEATURES
    df_features['temp_ewm_mean_7'] = df_features[target_col].ewm(span=7, min_periods=1).mean()
    df_features['temp_ewm_std_7'] = df_features[target_col].ewm(span=7, min_periods=1).std()

    return df_features

# Tạo features
print("\n===== TẠO FEATURES =====")
df_features = create_time_series_features(df, temp_col, date_col)

print(f"Số lượng features: {len(df_features.columns)}")
print(f"\nDanh sách features:")
for i, col in enumerate(df_features.columns, 1):
    print(f"{i}. {col}")

# Xử lý missing values
print(f"\n\nMissing values:")
missing_counts = df_features.isnull().sum()
if missing_counts.sum() > 0:
    print(missing_counts[missing_counts > 0].sort_values(ascending=False))
    df_features = df_features.dropna()
    print(f"\nShape sau khi xử lý missing values: {df_features.shape}")
else:
    print("Không có missing values!")

# ===== CHUẨN BỊ DỮ LIỆU CHO TRAINING =====

# Tách features và target
feature_columns = [col for col in df_features.columns if col not in [date_col, temp_col]]
X = df_features[feature_columns]
y = df_features[temp_col]

print(f"\n===== CHUẨN BỊ DỮ LIỆU =====")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Chia train/test theo thời gian (80/20)
split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print(f"\nTrain set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Train date range: {df_features[date_col].iloc[0]} to {df_features[date_col].iloc[split_idx-1]}")
print(f"Test date range: {df_features[date_col].iloc[split_idx]} to {df_features[date_col].iloc[-1]}")

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Chuyển về DataFrame để giữ tên cột
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_columns, index=X_test.index)

print("\n===== DỮ LIỆU ĐÃ CHUẨN BỊ XONG =====")
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")
print("\nMẫu dữ liệu X_train_scaled (5 dòng đầu):")
print(X_train_scaled.head())
print("\nThống kê y_train:")
print(y_train.describe())

# Kiểm tra không có NaN hoặc Inf
print("\n===== KIỂM TRA DỮ LIỆU =====")
print(f"NaN trong X_train_scaled: {X_train_scaled.isnull().sum().sum()}")
print(f"NaN trong X_test_scaled: {X_test_scaled.isnull().sum().sum()}")
print(f"Inf trong X_train_scaled: {np.isinf(X_train_scaled.values).sum()}")
print(f"Inf trong X_test_scaled: {np.isinf(X_test_scaled.values).sum()}")

print("\n✅ Feature Engineering hoàn thành!")
print("\nCác biến đã tạo:")
print("- X_train_scaled, X_test_scaled: Features đã chuẩn hóa")
print("- y_train, y_test: Target values")
print("- scaler: StandardScaler object")
print("- df_features: DataFrame đầy đủ với tất cả features")
print(f"- temp_col = '{temp_col}'")
print(f"- date_col = '{date_col}'")

## Training

### LinearRegression

In [ ]:
# ============================
# TRAIN LINEAR REGRESSION MODEL
# ============================

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np

# Khởi tạo mô hình
model_lr = LinearRegression()

# Huấn luyện mô hình
print("\n===== TRAINING LINEAR REGRESSION =====")
model_lr.fit(X_train_scaled, y_train)

# Dự đoán
y_pred_train = model_lr.predict(X_train_scaled)
y_pred_test = model_lr.predict(X_test_scaled)

# ============================
# ĐÁNH GIÁ MÔ HÌNH
# ============================

def evaluate_model(y_true, y_pred, model_name="Model"):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"\n📊 {model_name} Evaluation:")
    print(f"MAE  : {mae:.4f}")
    print(f"RMSE : {rmse:.4f}")
    print(f"R²   : {r2:.4f}")
    return mae, rmse, r2

train_scores = evaluate_model(y_train, y_pred_train, "Linear Regression (Train)")
test_scores = evaluate_model(y_test, y_pred_test, "Linear Regression (Test)")

# ============================
# VẼ BIỂU ĐỒ DỰ BÁO
# ============================
print(f"y_test shape: {y_test.shape}")
print(f"y_pred_test shape: {y_pred_test.shape}")

print("--- Chuẩn bị vẽ biểu đồ... ---")
# %matplotlib inline
plt.figure(figsize=(12, 5))
plt.plot(y_test.values, label="Actual", color="tab:blue")
plt.plot(y_pred_test, label="Predicted", color="tab:orange", alpha=0.8)
plt.title("📈 Linear Regression - Actual vs Predicted (Test set)")
plt.xlabel("Days")
plt.ylabel("Minimum Temperature (°C)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ============================
# KIỂM TRA CÁC HỆ SỐ (COEFFICIENTS)
# ============================

coef_df = pd.DataFrame({
    "Feature": X_train_scaled.columns,
    "Coefficient": model_lr.coef_
}).sort_values("Coefficient", key=lambda x: abs(x), ascending=False)

print("\n===== TOP 10 FEATURES QUAN TRỌNG =====")
print(coef_df.head(10))

# ============================
# KẾT LUẬN TẠM THỜI
# ============================

print("\n✅ Linear Regression training completed!")
print("- Mô hình tuyến tính đơn giản, dễ hiểu, nhanh huấn luyện.")
print("- Tuy nhiên, với dữ liệu Time Series có tính phi tuyến cao (seasonality rõ rệt), LR có thể dự báo chưa tốt ở các giai đoạn dao động mạnh.")
print("- Bước tiếp theo: huấn luyện Random Forest và XGBoost để so sánh.")



### Random Forest

In [ ]:
# ============================
# TRAIN & EVALUATE RANDOM FOREST
# ============================

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

# (Nếu ở trên bạn đã có evaluate_model thì đoạn dưới sẽ ghi đè cùng logic — không sao)
def evaluate_model(y_true, y_pred, model_name="Model"):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"\n📊 {model_name} Evaluation:")
    print(f"MAE  : {mae:.4f}")
    print(f"RMSE : {rmse:.4f}")
    print(f"R²   : {r2:.4f}")
    return mae, rmse, r2

# Khởi tạo RF (mặc định đã khá mạnh; có thể tune sau)
rf = RandomForestRegressor(
    n_estimators=400,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

print("\n===== TRAINING RANDOM FOREST =====")
rf.fit(X_train_scaled, y_train)

# Dự đoán
y_pred_train_rf = rf.predict(X_train_scaled)
y_pred_test_rf  = rf.predict(X_test_scaled)

# Đánh giá
train_scores_rf = evaluate_model(y_train, y_pred_train_rf, "Random Forest (Train)")
test_scores_rf  = evaluate_model(y_test,  y_pred_test_rf,  "Random Forest (Test)")

# Biểu đồ dự báo
plt.figure(figsize=(12, 5))
plt.plot(y_test.values, label="Actual", color="tab:blue")
plt.plot(y_pred_test_rf, label="RF Predicted", color="tab:green", alpha=0.9)
plt.title("🌲 Random Forest - Actual vs Predicted (Test set)")
plt.xlabel("Days")
plt.ylabel("Minimum Temperature (°C)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ============================
# FEATURE IMPORTANCE
# ============================

importances = rf.feature_importances_
fi_df = pd.DataFrame({
    "Feature": X_train_scaled.columns,
    "Importance": importances
}).sort_values("Importance", ascending=False)

print("\n===== TOP 15 FEATURE IMPORTANCE (RF) =====")
print(fi_df.head(15))

# Vẽ Top 15 features
topk = 15
plt.figure(figsize=(8, 6))
plt.barh(fi_df["Feature"].head(topk)[::-1], fi_df["Importance"].head(topk)[::-1])
plt.xlabel("Importance")
plt.title("Random Forest - Top Feature Importances")
plt.tight_layout()
plt.show()

print("\n✅ Random Forest training completed!")
print("- RF học tốt quan hệ phi tuyến/phi tham số, thường cải thiện lỗi so với Linear Regression.")
print("- Có thể tune nhanh: n_estimators, max_depth, min_samples_leaf để tối ưu thêm.")


### XGBoost

In [ ]:
# ============================
# TRAIN & EVALUATE XGBOOST + SUMMARY
# ============================

# Cài nếu thiếu: pip install xgboost
try:
    from xgboost import XGBRegressor
except ImportError as e:
    raise SystemExit("⚠️ Chưa cài xgboost. Chạy: pip install xgboost")

# Khởi tạo XGBoost (tham số an toàn, dễ hội tụ; có thể tune thêm)
xgb = XGBRegressor(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1,
    verbosity=0
)

print("\n===== TRAINING XGBOOST =====")
xgb.fit(X_train_scaled, y_train)

# Dự đoán
y_pred_train_xgb = xgb.predict(X_train_scaled)
y_pred_test_xgb  = xgb.predict(X_test_scaled)

# Đánh giá
train_scores_xgb = evaluate_model(y_train, y_pred_train_xgb, "XGBoost (Train)")
test_scores_xgb  = evaluate_model(y_test,  y_pred_test_xgb,  "XGBoost (Test)")

# Biểu đồ dự báo riêng XGBoost
plt.figure(figsize=(12, 5))
plt.plot(y_test.values, label="Actual", color="tab:blue")
plt.plot(y_pred_test_xgb, label="XGBoost Predicted", color="tab:red", alpha=0.9)
plt.title("⚡ XGBoost - Actual vs Predicted (Test set)")
plt.xlabel("Days")
plt.ylabel("Minimum Temperature (°C)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## So sánh tổng hợp 3 mô hình

In [ ]:
# ============================
# SO SÁNH TỔNG HỢP 3 MÔ HÌNH
# ============================

# đảm bảo đã có các biến dự đoán từ các bước trước:
# - y_pred_test (Linear Regression)
# - y_pred_test_rf (Random Forest)
# - y_pred_test_xgb (XGBoost)

import numpy as np
import pandas as pd

def metrics_dict(name, y_true, y_pred):
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    return {
        "Model": name,
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "R2": r2_score(y_true, y_pred)
    }

summary_rows = [
    metrics_dict("Linear Regression", y_test, y_pred_test),
    metrics_dict("Random Forest",     y_test, y_pred_test_rf),
    metrics_dict("XGBoost",           y_test, y_pred_test_xgb),
]
summary_df = pd.DataFrame(summary_rows).sort_values("RMSE").reset_index(drop=True)

print("\n=== So sánh tổng hợp (thấp hơn → tốt hơn) ===")
print(summary_df)

split_date = pd.to_datetime("1989-01-06")
test_idx = df['Date'] > split_date

# Lấy Date của tập test
test_dates = pd.to_datetime(df.loc[test_idx, 'Date']).reset_index(drop=True)

# Tạo DataFrame chuẩn để so sánh/zoom
y_test_df = pd.DataFrame({
    "Date":   test_dates,
    "Actual": y_test.values,              # giá trị thật của tập test
    "Linear Regression": y_pred_test,     # dự báo LR
    "Random Forest":     y_pred_test_rf,  # dự báo RF
    "XGBoost":           y_pred_test_xgb, # dự báo XGB
}).set_index("Date").sort_index()

import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.plot(y_test_df.index, y_test_df["Actual"], label="Actual", linewidth=2)
plt.plot(y_test_df.index, y_test_df["Linear Regression"], label="Linear Regression")
plt.plot(y_test_df.index, y_test_df["Random Forest"],     label="Random Forest")
plt.plot(y_test_df.index, y_test_df["XGBoost"],           label="XGBoost")
plt.title("So sánh dự báo trên tập Test (1989–1990)")
plt.xlabel("Date")
plt.ylabel("Minimum Temperature (°C)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

subset = y_test_df.loc["1989-07-01":"1989-07-07"]

plt.figure(figsize=(9,4))
plt.plot(subset.index, subset["Actual"], label="Actual", linewidth=2)
plt.plot(subset.index, subset["Linear Regression"], label="Linear Regression")
plt.plot(subset.index, subset["Random Forest"],     label="Random Forest")
plt.plot(subset.index, subset["XGBoost"],           label="XGBoost")
plt.title("Zoom 1 tuần: 1989-07-01 → 1989-07-07")
plt.xlabel("Date"); plt.ylabel("Minimum Temperature (°C)")
plt.legend(); plt.grid(True, alpha=0.3); plt.tight_layout(); plt.show()

subset = y_test_df.loc["1989-08-01":"1989-08-31"]

plt.figure(figsize=(10,4))
plt.plot(subset.index, subset["Actual"], label="Actual", linewidth=2)
plt.plot(subset.index, subset["Linear Regression"], label="Linear Regression")
plt.plot(subset.index, subset["Random Forest"],     label="Random Forest")
plt.plot(subset.index, subset["XGBoost"],           label="XGBoost")
plt.title("Zoom 1 tháng: 1989-08")
plt.xlabel("Date"); plt.ylabel("Minimum Temperature (°C)")
plt.legend(); plt.grid(True, alpha=0.3); plt.tight_layout(); plt.show()

# Diễn giải cho blog (in ra console)
best = summary_df.iloc[0]["Model"]
print(f"- {best} đang có RMSE thấp nhất trong 3 mô hình.")
print("- Linear Regression nhanh, dễ giải thích nhưng hạn chế với quan hệ phi tuyến.")
print("- Random Forest học phi tuyến tốt, ít cần chuẩn hóa, thường ổn định.")
print("- XGBoost thường thắng nhờ boosting, nhưng cần chỉnh tham số (n_estimators, learning_rate, max_depth...).")


## [WIP][Optional] Interactive Dashboard
Phần này mình tìm hiểu thêm

My document:
- [Data Analyst & Create Interactive Dashboard for Temperature Data](https://concrete-tray-472.notion.site/Data-Analyst-Create-Interactive-Dashboard-for-Temperature-Data-2900730a967380c8a8f3cb7f8463b01d)

In [ ]:
%pip -q install -U gradio pandas numpy scikit-learn xgboost plotly shap matplotlib statsmodels
%pip -q install nest_asyncio

In [ ]:
# --- 1. IMPORT & CẤU HÌNH ---
%pip install -U gradio pandas numpy scikit-learn xgboost plotly shap matplotlib statsmodels
%pip install nest_asyncio

import gradio as gr
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import shap
import matplotlib.pyplot as plt
from functools import lru_cache
import warnings
import nest_asyncio

# Áp dụng bản vá cho asyncio, rất quan trọng khi chạy Gradio trong Jupyter/Colab
nest_asyncio.apply()
warnings.filterwarnings('ignore')


# --- 2. CÁC HÀM BACKEND ---

def create_time_series_features(df):
    df_feat = df.copy()
    df_feat['month'] = df_feat.index.month
    df_feat['dayofyear'] = df_feat.index.dayofyear
    df_feat['weekofyear'] = df_feat.index.isocalendar().week.astype(int)
    df_feat['dayofweek'] = df_feat.index.dayofweek
    df_feat['quarter'] = df_feat.index.quarter
    df_feat['year'] = df_feat.index.year
    df_feat['lag_1'] = df_feat['temp'].shift(1)
    df_feat['lag_7'] = df_feat['temp'].shift(7)
    df_feat['rolling_mean_7'] = df_feat['temp'].shift(1).rolling(window=7).mean()
    df_feat = df_feat.dropna()
    return df_feat

def train_model_and_predict(df_raw):
    df_featured = create_time_series_features(df_raw)
    split_date = '1990-01-01'
    train = df_featured.loc[df_featured.index < split_date]
    test = df_featured.loc[df_featured.index >= split_date]
    if train.empty or test.empty:
        raise ValueError("Không đủ dữ liệu để chia tập train/test (cần dữ liệu cả trước và sau 1990-01-01).")
    X_train, y_train = train.drop('temp', axis=1), train['temp']
    model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.05, objective='reg:squarederror', eval_metric='rmse')
    model.fit(X_train, y_train, verbose=False)
    predictions = model.predict(test.drop('temp', axis=1))
    results_df = pd.DataFrame({'Actual': test['temp'], 'Predicted': predictions})
    results_df['Error'] = results_df['Actual'] - results_df['Predicted']
    return model, X_train, df_featured, results_df

# --- 3. CÁC HÀM TƯƠNG TÁC VỚI GIAO DIỆN GRADIO ---

def process_uploaded_file(uploaded_file):
    if uploaded_file is None:
        return (gr.update(visible=False), None, gr.update(value="Vui lòng tải lên file CSV.", visible=True), None, None)
    try:
        df_raw = pd.read_csv(uploaded_file.name)
        df_raw.columns = ['date', 'temp']
        df_raw = df_raw[~df_raw['temp'].str.contains('[^0-9.-]', na=False)]
        df_raw['date'] = pd.to_datetime(df_raw['date'])
        df_raw['temp'] = df_raw['temp'].astype(float)
        df_raw = df_raw.sort_values('date').set_index('date')
        model, X_train, df_featured, results_df = train_model_and_predict(df_raw)
        processed_data = {"model": model, "X_train": X_train, "df_featured": df_featured, "results_df": results_df}
        min_date, max_date = results_df.index.min().strftime('%Y-%m-%d'), results_df.index.max().strftime('%Y-%m-%d')
        return (gr.update(visible=True), processed_data, gr.update(value="✅ **File đã xử lý!**", visible=True), gr.update(value=min_date), gr.update(value=max_date))
    except Exception as e:
        return (gr.update(visible=False), None, gr.update(value=f"❌ **Lỗi:** {e}", visible=True), None, None)

def update_all_tabs(start_date, end_date, processed_data):
    if processed_data is None: return tuple(["N/A"] * 3 + [None] * 9)
    try:
        results_df, df_featured = processed_data["results_df"], processed_data["df_featured"]
        start_ts, end_ts = pd.to_datetime(start_date), pd.to_datetime(end_date)
        filtered_results = results_df[(results_df.index >= start_ts) & (results_df.index <= end_ts)]
        if filtered_results.empty: raise ValueError("Không có dữ liệu trong khoảng thời gian này.")

        mae = mean_absolute_error(filtered_results['Actual'], filtered_results['Predicted'])
        rmse = np.sqrt(mean_squared_error(filtered_results['Actual'], filtered_results['Predicted']))
        r2 = r2_score(filtered_results['Actual'], filtered_results['Predicted'])
        fig_actual_vs_pred = px.line(filtered_results, y=['Actual', 'Predicted'], title="Thực tế vs. Dự báo")
        rolling_mae = filtered_results['Error'].abs().rolling(window=7).mean()
        fig_mae_time = px.line(rolling_mae, title="MAE trượt 7 ngày")
        mean_error, std_error, max_error = filtered_results['Error'].mean(), filtered_results['Error'].std(), filtered_results['Error'].abs().max()
        fig_error_time = px.line(filtered_results, y='Error', title="Sai số theo thời gian")
        fig_hist_error = px.histogram(filtered_results, x='Error', nbins=30, title="Phân phối Sai số")
        fig_scatter = px.scatter(filtered_results, x='Actual', y='Predicted', title="Tương quan Thực tế vs. Dự báo", trendline='ols')
        fig_drift = go.Figure()
        fig_drift.add_trace(go.Histogram(x=df_featured.loc[df_featured.index < '1990-01-01', 'temp'], name='Training Data', histnorm='probability density', opacity=0.7))
        fig_drift.add_trace(go.Histogram(x=filtered_results['Actual'], name='Current Data', histnorm='probability density', opacity=0.7))
        fig_drift.update_layout(barmode='overlay', title_text='So sánh Phân phối Dữ liệu')

        return (f"{mae:.2f}°C", f"{rmse:.2f}°C", f"{r2:.2f}", fig_actual_vs_pred, fig_mae_time, f"{mean_error:.2f}°C", f"{std_error:.2f}°C", f"{max_error:.2f}°C", fig_error_time, fig_hist_error, fig_scatter, fig_drift)
    except Exception as e:
        error_fig = go.Figure(layout={"title": f"Lỗi: {e}"})
        return tuple(["Lỗi"] * 3 + [error_fig] * 9)

# ======================================================================
# >>> PHẦN SỬA LỖI QUAN TRỌNG NHẤT <<<
def handle_tab_selection(processed_data, evt: gr.SelectData):
    """
    Hàm xử lý sự kiện chọn tab.
    Nó CHỈ tính toán SHAP khi tab 'Diễn giải Mô hình' được chọn.
    """
    if evt.index == 3:  # Index của Tab 'Diễn giải Mô hình'
        if processed_data is None:
            fig, ax = plt.subplots(); ax.text(0.5, 0.5, "Vui lòng tải và xử lý file trước.", ha='center'); return fig
        try:
            model = processed_data["model"]
            # Lấy DataFrame từ state và KHÔI PHỤC KIỂU DỮ LIỆU
            X_train = processed_data["X_train"].astype(float)

            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X_train)

            fig, ax = plt.subplots()
            shap.summary_plot(shap_values, X_train, plot_type="bar", show=False)
            plt.tight_layout()
            return fig
        except Exception as e:
            fig, ax = plt.subplots(); ax.text(0.5, 0.5, f"Lỗi khi tạo biểu đồ SHAP:\n{e}", ha='center', wrap=True); return fig
    # Nếu không phải tab SHAP, không làm gì cả
    return gr.update()
# >>> KẾT THÚC PHẦN SỬA LỖI <<<
# ======================================================================

# --- 4. XÂY DỰNG GIAO DIỆN ---
with gr.Blocks(theme='soft', title="Dashboard Dự báo Nhiệt độ") as demo:
    processed_data_state = gr.State()
    gr.Markdown("# 🌤️ Dashboard Giám sát Dự báo Nhiệt độ Tối thiểu")
    file_uploader = gr.File(label="Bước 1: Tải lên file 'timeseries_daily-minimum-temperatures.csv'", file_types=[".csv"])
    status_markdown = gr.Markdown("Vui lòng tải lên file CSV để bắt đầu.")
    with gr.Column(visible=False) as main_dashboard:
        gr.Markdown("---")
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("#### Bước 2: Bộ lọc")
                start_date_picker = gr.Textbox(label="Ngày bắt đầu (YYYY-MM-DD)")
                end_date_picker = gr.Textbox(label="Ngày kết thúc (YYYY-MM-DD)")
                update_button = gr.Button("Cập nhật Dashboard", variant="primary")
            with gr.Column(scale=3):
                with gr.Tabs() as tabs:
                    with gr.TabItem("📊 1. Tổng quan Hiệu suất", id=0):
                        with gr.Row(): mae_kpi, rmse_kpi, r2_kpi = [gr.Textbox(label=l, interactive=False) for l in ["MAE", "RMSE", "R² Score"]]
                        plot_actual_pred = gr.Plot()
                        plot_mae_time = gr.Plot()
                    with gr.TabItem("🔍 2. Phân tích Lỗi", id=1):
                        with gr.Row(): mean_error_kpi, std_error_kpi, max_error_kpi = [gr.Textbox(label=l, interactive=False) for l in ["Mean Error (Bias)", "Std Dev of Error", "Max Absolute Error"]]
                        plot_error_time = gr.Plot()
                        with gr.Row(): plot_hist_error = gr.Plot(); plot_scatter = gr.Plot()
                    with gr.TabItem("📉 3. Data & Prediction Drift", id=2):
                        plot_drift = gr.Plot()
                    with gr.TabItem("🧠 6. Diễn giải Mô hình", id=3):
                        plot_shap = gr.Plot()
    # --- 5. KẾT NỐI TƯƠNG TÁC ---
    all_outputs = [mae_kpi, rmse_kpi, r2_kpi, plot_actual_pred, plot_mae_time, mean_error_kpi, std_error_kpi, max_error_kpi, plot_error_time, plot_hist_error, plot_scatter, plot_drift]
    file_uploader.upload(fn=process_uploaded_file, inputs=file_uploader, outputs=[main_dashboard, processed_data_state, status_markdown, start_date_picker, end_date_picker])
    update_button.click(fn=update_all_tabs, inputs=[start_date_picker, end_date_picker, processed_data_state], outputs=all_outputs)
    tabs.select(fn=handle_tab_selection, inputs=processed_data_state, outputs=plot_shap)
# --- 6. KHỞI CHẠY DASHBOARD ---
demo.launch(share=True, inline=False, debug=True)